In [1]:
from corpus_characterizer import generator_chunker
import numpy as np
import os
# from sklearn.metrics import mean_squared_error,mean_absolute_error, median_absolute_error, mean_squared_log_error, explained_variance_score, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, explained_variance_score, \
    r2_score
import pandas as pd
from scipy.stats import describe, kurtosistest, skewtest, normaltest
from Conv1D_LSTM_Ensemble import pair_generator_1dconv_lstm_bagged
from Conv1D_ActivationSearch_BigLoop import pair_generator_1dconv_lstm #NOT BAGGED
from AuxRegressor import create_testing_set, create_training_set
import matplotlib.pyplot as plt 

# @@@@@@@@@@@@@@ RELATIVE PATHS @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Base_Path = "./"
image_path = "./images/"
train_path = "./train/"
test_path = "./test/"
analysis_path = "./analysis/"
chunker_path = analysis_path + "chunker/"
preds_path = analysis_path + "preds/"

#create the data-pair filenames (using zip), use the helper methods
train_set_filenames = create_training_set()
test_set_filenames = create_testing_set()
print(train_set_filenames)
print(test_set_filenames)


Using Theano backend.


after shuffling: [('sequence_2b_484_5_fv1c.npy', 'sequence_2b_484_5_label_fv1c.npy'), ('sequence_1l_413_4_fv1c.npy', 'sequence_1l_413_4_label_fv1c.npy'), ('sequence_1d_161_2_fv1c.npy', 'sequence_1d_161_2_label_fv1c.npy'), ('sequence_2c_551_2_fv1c.npy', 'sequence_2c_551_2_label_fv1c.npy'), ('sequence_1l_411_2_fv1c.npy', 'sequence_1l_411_2_label_fv1c.npy'), ('sequence_3a_561_2_fv1c.npy', 'sequence_3a_561_2_label_fv1c.npy'), ('sequence_3a_690_12_fv1c.npy', 'sequence_3a_690_12_label_fv1c.npy'), ('sequence_1c_105_6_fv1c.npy', 'sequence_1c_105_6_label_fv1c.npy'), ('sequence_1d_197_8_fv1c.npy', 'sequence_1d_197_8_label_fv1c.npy'), ('sequence_3a_715_17_fv1c.npy', 'sequence_3a_715_17_label_fv1c.npy'), ('sequence_1b_15_6_fv1c.npy', 'sequence_1b_15_6_label_fv1c.npy'), ('sequence_2b_485_6_fv1c.npy', 'sequence_2b_485_6_label_fv1c.npy'), ('sequence_3a_600_2_fv1c.npy', 'sequence_3a_600_2_label_fv1c.npy'), ('sequence_3a_664_6_fv1c.npy', 'sequence_3a_664_6_label_fv1c.npy'), ('sequence_3a_646_8_fv1c.npy

In [2]:
i = 0 
unique_load_sequences = [] #fill these with the 4-pairs as a numpy array
unique_load_sequences_aslists = []
sequences_4pairs_list = []
sequences_not4pairs_list = []
filename_loadseq_dict_seqidkey = {} #key is the sequence ID, value is its position in the list. 
four_pairs_condition = False
print("train_set size: ",len(train_set_filenames))
for files in train_set_filenames:
    #print("filename", files)
    i += 1
    data_load_path = train_path + '/data/' + files[0]
    label_load_path = train_path + '/label/' + files[1]
    # print("data/label load path: {} \n {}".format(data_load_path,label_load_path))
    train_array = np.load(data_load_path)
    train_label_array = np.load(label_load_path)[:, 1:]
    
    #check if the repeating unit is actually 4 tuples. 
    #four_pair_condition = (train_array[0:5,-2] == train_array[4:8,-2]) and (train_array[0:5,-1] == train_array[4:8,-1]) error. 
    #numpy version 
    four_pair_condition = np.array_equal(train_array[0:4,-2:],train_array[4:8,-2:]) 

    
    if four_pair_condition == True:
        #print(str(files[0])," is in.")
        load_sequence = train_array[0:4,-2:]
        #if len(unique_load_sequences) != 0 :
        array_equality = False
        if len(unique_load_sequences) == 0:
            unique_load_sequences.append(load_sequence)
            #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence)) #need to be flattened into a list to make it hashable for the list-operators to work
            unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
        if len(unique_load_sequences) != 0:
            for load_seq in unique_load_sequences:
                if np.array_equal(load_seq,load_sequence) == True:
                    #print("equal array found in list?", np.array_equal(load_seq,load_sequence))
                    array_equality = True
            if array_equality == False:
                unique_load_sequences.append(load_sequence) #the load types
                unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
            filename_loadseq_dict_seqidkey[str(files[0])[:-4]] = unique_load_sequences_aslists.index(str(np.ndarray.flatten(load_sequence)))
        sequences_4pairs_list.append(str(files[0]))
    if four_pair_condition == False:
        sequences_not4pairs_list.append(str(files[0]))
        
    
    #print("train_array's last two columns",train_array[0:20,-2:])
#unique_load_sequences = set(unique_load_sequences)
#print("there are", len(unique_load_sequences), " load sequences")
#print("the unique load sequences are:", unique_load_sequences)
print("sequences with 4-pair repeating units: ",len(sequences_4pairs_list))
print("sequences WITHOUT 4-pair repeating units: ",sequences_not4pairs_list)

('train_set size: ', 611)
('sequences with 4-pair repeating units: ', 357)
('sequences WITHOUT 4-pair repeating units: ', ['sequence_1l_413_4_fv1c.npy', 'sequence_1l_411_2_fv1c.npy', 'sequence_3a_561_2_fv1c.npy', 'sequence_3a_690_12_fv1c.npy', 'sequence_3a_715_17_fv1c.npy', 'sequence_3a_600_2_fv1c.npy', 'sequence_3a_664_6_fv1c.npy', 'sequence_3a_646_8_fv1c.npy', 'sequence_1k_337_8_fv1c.npy', 'sequence_3a_618_20_fv1c.npy', 'sequence_3a_570_11_fv1c.npy', 'sequence_1l_394_5_fv1c.npy', 'sequence_1l_404_5_fv1c.npy', 'sequence_3a_703_5_fv1c.npy', 'sequence_1k_321_2_fv1c.npy', 'sequence_3a_584_6_fv1c.npy', 'sequence_1k_292_3_fv1c.npy', 'sequence_1l_371_2_fv1c.npy', 'sequence_1l_402_3_fv1c.npy', 'sequence_3a_650_12_fv1c.npy', 'sequence_1k_344_5_fv1c.npy', 'sequence_1l_401_2_fv1c.npy', 'sequence_3a_718_20_fv1c.npy', 'sequence_3a_661_3_fv1c.npy', 'sequence_1l_364_5_fv1c.npy', 'sequence_1k_357_8_fv1c.npy', 'sequence_1k_341_2_fv1c.npy', 'sequence_1l_397_8_fv1c.npy', 'sequence_3a_582_4_fv1c.npy', '

In [3]:
twelve_pair_condition = np.array_equal(train_array[0:12,-2:],train_array[12:24,-2:]) 
sequences_12pairs_list = []
sequences_not4_not12_pairs_list = []

#gotta run the 4-pair filter first to make sure the 12-pair subset list is ok. Hence the separate loop. 
for unusual_seqs in sequences_not4pairs_list: #12 repeating units. 
    data_load_path = train_path + '/data/' + unusual_seqs
    train_array = np.load(data_load_path)
    
    if twelve_pair_condition == True:
        #print(str(unusual_seqs)," is in.")
        load_sequence = train_array[0:12,-2:]
        #if len(unique_load_sequences) != 0 :
        array_equality = False
        if len(unique_load_sequences) == 0:
            unique_load_sequences.append(load_sequence)
            #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence))
            unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
        if len(unique_load_sequences) != 0:
            for load_seq in unique_load_sequences:
                if np.array_equal(load_seq,load_sequence) == True:
                    #print("equal array found in list?", np.array_equal(load_seq,load_sequence))
                    array_equality = True
            if array_equality == False:
                unique_load_sequences.append(load_sequence) #the load types
                #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence))
                unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
                                        
        sequences_12pairs_list.append(str(unusual_seqs))
        filename_loadseq_dict_seqidkey[str(unusual_seqs)[:-4]] = unique_load_sequences_aslists.index(str(np.ndarray.flatten(load_sequence)))
    if twelve_pair_condition == False:
        sequences_not4_not12_pairs_list.append(str(unusual_seqs))

In [4]:
print("There are ",len(sequences_12pairs_list), " sequences with 12-pairs")        
print("There are ", len(sequences_not4_not12_pairs_list), "sequences that have neither 4 nor 12 repeating units")
print("seqs that aren't 4 or 12: ",sequences_not4_not12_pairs_list)
print("there are ",len(unique_load_sequences), " different load sequences in the ", len(train_set_filenames), "sequences in the training set")
#print("unique load seqs", unique_load_sequences)

('There are ', 254, ' sequences with 12-pairs')
('There are ', 0, 'sequences that have neither 4 nor 12 repeating units')
("seqs that aren't 4 or 12: ", [])
('there are ', 27, ' different load sequences in the ', 611, 'sequences in the training set')


In [5]:
#just query this. e.g. print dict[sequence id] 
print("the dict with seq id as key: ", filename_loadseq_dict_seqidkey)

('the dict with seq id as key: ', {'sequence_1k_294_5_fv1c': 23, 'sequence_1d_161_2_fv1c': 1, 'sequence_1c_61_2_fv1c': 7, 'sequence_1d_269_20_fv1c': 5, 'sequence_3a_676_18_fv1c': 12, 'sequence_1k_315_6_fv1c': 3, 'sequence_1d_172_13_fv1c': 1, 'sequence_1c_113_4_fv1c': 5, 'sequence_3a_604_6_fv1c': 16, 'sequence_1d_207_18_fv1c': 3, 'sequence_3a_586_8_fv1c': 23, 'sequence_1c_104_5_fv1c': 0, 'sequence_1d_136_7_fv1c': 4, 'sequence_2b_466_7_fv1c': 1, 'sequence_1d_126_7_fv1c': 7, 'sequence_2b_450_1_fv1c': 4, 'sequence_1d_246_27_fv1c': 0, 'sequence_3a_564_5_fv1c': 13, 'sequence_1d_263_14_fv1c': 5, 'sequence_2c_557_8_fv1c': 2, 'sequence_1l_373_4_fv1c': 24, 'sequence_2b_483_4_fv1c': 0, 'sequence_1k_333_4_fv1c': 17, 'sequence_3a_623_5_fv1c': 3, 'sequence_1d_141_12_fv1c': 4, 'sequence_3a_649_11_fv1c': 21, 'sequence_3a_620_2_fv1c': 3, 'sequence_1k_336_7_fv1c': 17, 'sequence_3a_645_7_fv1c': 18, 'sequence_1k_330_1_fv1c': 17, 'sequence_3a_626_8_fv1c': 3, 'sequence_2c_524_5_fv1c': 10, 'sequence_1l_422_3

In [6]:
#make the reverse. load-sequence as key, sequence ID (as lists) as values. 
#dict[key].append(value as list)
filename_loadseq_dict_loadlistposkey = {}

for key,value in filename_loadseq_dict_seqidkey.iteritems():
    print("key",key,"value",value)
    load_sequence_aslist = str(unique_load_sequences_aslists[value])
    #this arrangement below is necessary because dict has to have a list as its value. 
    #casting the list-cast ndarray as a string means.. parsing it back to numpy requires some creativity. 
    if filename_loadseq_dict_loadlistposkey.get(str(load_sequence_aslist)) != None:
        filename_loadseq_dict_loadlistposkey[str(load_sequence_aslist)].append(key)
    if filename_loadseq_dict_loadlistposkey.get(str(load_sequence_aslist)) == None: 
        content_list = []
        content_list.append(key)
        filename_loadseq_dict_loadlistposkey[str(load_sequence_aslist)] = content_list

        
#print(filename_loadseq_dict_loadlistposkey)
# fl_llpk_df = pd.DataFrame.from_dict(filename_loadseq_dict_loadlistposkey,orient='index')
# fl_sik_df = pd.DataFrame.from_dict(filename_loadseq_dict_seqidkey,orient='index')
# fl_llpk_df.to_csv(analysis_path + "file_loadseq_loadlistkey_TRAIN.csv")
# fl_sik_df.to_csv(analysis_path + "file_loadseq_seqid_key_TRAIN.csv")


('key', 'sequence_1k_294_5_fv1c', 'value', 23)
('key', 'sequence_1d_161_2_fv1c', 'value', 1)
('key', 'sequence_1c_61_2_fv1c', 'value', 7)
('key', 'sequence_1d_269_20_fv1c', 'value', 5)
('key', 'sequence_3a_676_18_fv1c', 'value', 12)
('key', 'sequence_1k_315_6_fv1c', 'value', 3)
('key', 'sequence_1d_172_13_fv1c', 'value', 1)
('key', 'sequence_1c_113_4_fv1c', 'value', 5)
('key', 'sequence_3a_604_6_fv1c', 'value', 16)
('key', 'sequence_1d_207_18_fv1c', 'value', 3)
('key', 'sequence_3a_586_8_fv1c', 'value', 23)
('key', 'sequence_1c_104_5_fv1c', 'value', 0)
('key', 'sequence_1d_136_7_fv1c', 'value', 4)
('key', 'sequence_2b_466_7_fv1c', 'value', 1)
('key', 'sequence_1d_126_7_fv1c', 'value', 7)
('key', 'sequence_2b_450_1_fv1c', 'value', 4)
('key', 'sequence_1d_246_27_fv1c', 'value', 0)
('key', 'sequence_3a_564_5_fv1c', 'value', 13)
('key', 'sequence_1d_263_14_fv1c', 'value', 5)
('key', 'sequence_2c_557_8_fv1c', 'value', 2)
('key', 'sequence_1l_373_4_fv1c', 'value', 24)
('key', 'sequence_2b_48

In [7]:
#now let's play with the test set. 

i = 0 
unique_load_sequences = [] #fill these with the 4-pairs as a numpy array
unique_load_sequences_aslists = []
sequences_4pairs_list = []
sequences_not4pairs_list = []
filename_loadseq_dict_seqidkey = {} #key is the sequence ID, value is its position in the list. 
four_pairs_condition = False
print("test_set size: ",len(test_set_filenames))
for files in test_set_filenames:
    #print("filename", files)
    i += 1
    data_load_path = test_path + '/data/' + files[0]
    label_load_path = test_path + '/label/' + files[1]
    # print("data/label load path: {} \n {}".format(data_load_path,label_load_path))
    test_array = np.load(data_load_path)
    test_label_array = np.load(label_load_path)[:, 1:]
    
    #check if the repeating unit is actually 4 tuples. 
    #four_pair_condition = (test_array[0:5,-2] == test_array[4:8,-2]) and (test_array[0:5,-1] == test_array[4:8,-1]) error. 
    #numpy version 
    four_pair_condition = np.array_equal(test_array[0:4,-2:],test_array[4:8,-2:]) 

    
    if four_pair_condition == True:
        #print(str(files[0])," is in.")
        load_sequence = test_array[0:4,-2:]
        #if len(unique_load_sequences) != 0 :
        array_equality = False
        if len(unique_load_sequences) == 0:
            unique_load_sequences.append(load_sequence)
            #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence)) #need to be flattened into a list to make it hashable for the list-operators to work
            unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
        if len(unique_load_sequences) != 0:
            for load_seq in unique_load_sequences:
                if np.array_equal(load_seq,load_sequence) == True:
                    #print("equal array found in list?", np.array_equal(load_seq,load_sequence))
                    array_equality = True
            if array_equality == False:
                unique_load_sequences.append(load_sequence) #the load types
                #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence))
                unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
            filename_loadseq_dict_seqidkey[str(files[0])[:-4]] = unique_load_sequences_aslists.index(str(np.ndarray.flatten(load_sequence)))
        sequences_4pairs_list.append(str(files[0]))
    if four_pair_condition == False:
        sequences_not4pairs_list.append(str(files[0]))
        
    
    #print("test_array's last two columns",test_array[0:20,-2:])
#unique_load_sequences = set(unique_load_sequences)
#print("there are", len(unique_load_sequences), " load sequences")
#print("the unique load sequences are:", unique_load_sequences)
print("sequences with 4-pair repeating units: ",len(sequences_4pairs_list))
print("sequences WITHOUT 4-pair repeating units: ",sequences_not4pairs_list)

twelve_pair_condition = np.array_equal(test_array[0:12,-2:],test_array[12:24,-2:]) 
sequences_12pairs_list = []
sequences_not4_not12_pairs_list = []

#gotta run the 4-pair filter first to make sure the 12-pair subset list is ok. Hence the separate loop. 
for unusual_seqs in sequences_not4pairs_list: #12 repeating units. 
    data_load_path = test_path + '/data/' + unusual_seqs
    test_array = np.load(data_load_path)
    
    if twelve_pair_condition == True:
        #print(str(unusual_seqs)," is in.")
        load_sequence = test_array[0:12,-2:]
        #if len(unique_load_sequences) != 0 :
        array_equality = False
        if len(unique_load_sequences) == 0:
            unique_load_sequences.append(load_sequence)
            #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence))
            unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
        if len(unique_load_sequences) != 0:
            for load_seq in unique_load_sequences:
                if np.array_equal(load_seq,load_sequence) == True:
                    #print("equal array found in list?", np.array_equal(load_seq,load_sequence))
                    array_equality = True
            if array_equality == False:
                unique_load_sequences.append(load_sequence) #the load types
                #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence))
                unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
        sequences_12pairs_list.append(str(unusual_seqs))
        filename_loadseq_dict_seqidkey[str(unusual_seqs)[:-4]] = unique_load_sequences_aslists.index(str(np.ndarray.flatten(load_sequence)))
    if twelve_pair_condition == False:
        sequences_not4_not12_pairs_list.append(str(unusual_seqs))

print("There are ",len(sequences_12pairs_list), " sequences with 12-pairs")        
print("There are ", len(sequences_not4_not12_pairs_list), "sequences that have neither 4 nor 12 repeating units")
print("seqs that aren't 4 or 12: ",sequences_not4_not12_pairs_list)
print("there are ",len(unique_load_sequences), " different load sequences in the ", len(test_set_filenames), "sequences in the training set")
#print("unique load seqs", unique_load_sequences)

for key,value in filename_loadseq_dict_seqidkey.iteritems():
    print("key",key,"value",value)
    load_sequence_aslist = str(unique_load_sequences_aslists[value])
    #this arrangement below is necessary because dict has to have a list as its value. 
    #casting the list-cast ndarray as a string means.. parsing it back to numpy requires some creativity. 
    if filename_loadseq_dict_loadlistposkey.get(str(load_sequence_aslist)) != None:
        filename_loadseq_dict_loadlistposkey[str(load_sequence_aslist)].append(key)
    if filename_loadseq_dict_loadlistposkey.get(str(load_sequence_aslist)) == None: 
        content_list = []
        content_list.append(key)
        filename_loadseq_dict_loadlistposkey[str(load_sequence_aslist)] = content_list

print(filename_loadseq_dict_loadlistposkey)
fl_llpk_test_df = pd.DataFrame.from_dict(filename_loadseq_dict_loadlistposkey,orient='index')
fl_sik_test_df = pd.DataFrame.from_dict(filename_loadseq_dict_seqidkey,orient='index')

fl_llpk_test_df.to_csv(analysis_path + "file_loadseq_loadlistkey_TEST.csv")
fl_sik_test_df.to_csv(analysis_path + "file_loadseq_seqid_key_TEST.csv")

#append the string of the loadseq #file_loadseq_seqid_key_TEST.csv
loadseq_loadlist_df = pd.DataFrame(unique_load_sequences_aslists,columns=['loadseq'])
loadseq_loadlist_df.to_csv(analysis_path + "load_list_code.csv")
# for item in loadseq_loadlist_df:
#     loadseq_loadlist_df.loc[]


('test_set size: ', 108)
('sequences with 4-pair repeating units: ', 63)
('sequences WITHOUT 4-pair repeating units: ', ['sequence_1l_419_10_fv1c.npy', 'sequence_3a_628_10_fv1c.npy', 'sequence_1l_428_9_fv1c.npy', 'sequence_1k_308_9_fv1c.npy', 'sequence_1k_358_9_fv1c.npy', 'sequence_3a_587_9_fv1c.npy', 'sequence_3a_647_9_fv1c.npy', 'sequence_1k_328_9_fv1c.npy', 'sequence_1k_288_9_fv1c.npy', 'sequence_3a_707_9_fv1c.npy', 'sequence_1l_379_10_fv1c.npy', 'sequence_1l_368_9_fv1c.npy', 'sequence_3a_648_10_fv1c.npy', 'sequence_1k_339_10_fv1c.npy', 'sequence_1l_398_9_fv1c.npy', 'sequence_1l_408_9_fv1c.npy', 'sequence_1k_348_9_fv1c.npy', 'sequence_3a_568_9_fv1c.npy', 'sequence_1l_438_9_fv1c.npy', 'sequence_1k_329_10_fv1c.npy', 'sequence_1k_349_10_fv1c.npy', 'sequence_1k_359_10_fv1c.npy', 'sequence_1k_299_10_fv1c.npy', 'sequence_1l_378_9_fv1c.npy', 'sequence_3a_588_10_fv1c.npy', 'sequence_3a_708_10_fv1c.npy', 'sequence_1l_429_10_fv1c.npy', 'sequence_1l_418_9_fv1c.npy', 'sequence_3a_607_9_fv1c.npy

In [8]:
#run the train set again to make the combined (test+train) dict; most of the lists aren't reset. 

i = 0 
# unique_load_sequences = [] #fill these with the 4-pairs as a numpy array
# unique_load_sequences_aslists = []
sequences_4pairs_list = []
sequences_not4pairs_list = []
# filename_loadseq_dict_seqidkey = {} #key is the sequence ID, value is its position in the list. 
four_pairs_condition = False
print("train_set size: ",len(train_set_filenames))
for files in train_set_filenames:
    #print("filename", files)
    i += 1
    data_load_path = train_path + '/data/' + files[0]
    label_load_path = train_path + '/label/' + files[1]
    # print("data/label load path: {} \n {}".format(data_load_path,label_load_path))
    train_array = np.load(data_load_path)
    train_label_array = np.load(label_load_path)[:, 1:]
    
    #check if the repeating unit is actually 4 tuples. 
    #four_pair_condition = (train_array[0:5,-2] == train_array[4:8,-2]) and (train_array[0:5,-1] == train_array[4:8,-1]) error. 
    #numpy version 
    four_pair_condition = np.array_equal(train_array[0:4,-2:],train_array[4:8,-2:]) 

    
    if four_pair_condition == True:
        #print(str(files[0])," is in.")
        load_sequence = train_array[0:4,-2:]
        #if len(unique_load_sequences) != 0 :
        array_equality = False
        if len(unique_load_sequences) == 0:
            unique_load_sequences.append(load_sequence)
            #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence)) #need to be flattened into a list to make it hashable for the list-operators to work
            unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
        if len(unique_load_sequences) != 0:
            for load_seq in unique_load_sequences:
                if np.array_equal(load_seq,load_sequence) == True:
                    #print("equal array found in list?", np.array_equal(load_seq,load_sequence))
                    array_equality = True
            if array_equality == False:
                unique_load_sequences.append(load_sequence) #the load types
                #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence))
                unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
            filename_loadseq_dict_seqidkey[str(files[0])] = unique_load_sequences_aslists.index(str(np.ndarray.flatten(load_sequence)))
        sequences_4pairs_list.append(str(files[0]))
    if four_pair_condition == False:
        sequences_not4pairs_list.append(str(files[0]))
        
    
    #print("train_array's last two columns",train_array[0:20,-2:])
#unique_load_sequences = set(unique_load_sequences)
#print("there are", len(unique_load_sequences), " load sequences")
#print("the unique load sequences are:", unique_load_sequences)
print("sequences with 4-pair repeating units: ",len(sequences_4pairs_list))
print("sequences WITHOUT 4-pair repeating units: ",sequences_not4pairs_list)

twelve_pair_condition = np.array_equal(train_array[0:12,-2:],train_array[12:24,-2:]) 
sequences_12pairs_list = []
sequences_not4_not12_pairs_list = []

#gotta run the 4-pair filter first to make sure the 12-pair subset list is ok. Hence the separate loop. 
for unusual_seqs in sequences_not4pairs_list: #12 repeating units. 
    data_load_path = train_path + '/data/' + unusual_seqs
    train_array = np.load(data_load_path)
    
    if twelve_pair_condition == True:
        #print(str(unusual_seqs)," is in.")
        load_sequence = train_array[0:12,-2:]
        #if len(unique_load_sequences) != 0 :
        array_equality = False
        if len(unique_load_sequences) == 0:
            unique_load_sequences.append(load_sequence)
            #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence))
            unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
        if len(unique_load_sequences) != 0:
            for load_seq in unique_load_sequences:
                if np.array_equal(load_seq,load_sequence) == True:
                    #print("equal array found in list?", np.array_equal(load_seq,load_sequence))
                    array_equality = True
            if array_equality == False:
                unique_load_sequences.append(load_sequence) #the load types
                #unique_load_sequences_aslists.append(np.ndarray.tolist(load_sequence))
                unique_load_sequences_aslists.append(str(np.ndarray.flatten(load_sequence)))
        sequences_12pairs_list.append(str(unusual_seqs))
        filename_loadseq_dict_seqidkey[str(unusual_seqs)] = unique_load_sequences_aslists.index(str(np.ndarray.flatten(load_sequence)))
    if twelve_pair_condition == False:
        sequences_not4_not12_pairs_list.append(str(unusual_seqs))

print("There are ",len(sequences_12pairs_list), " sequences with 12-pairs")        
print("There are ", len(sequences_not4_not12_pairs_list), "sequences that have neither 4 nor 12 repeating units")
print("seqs that aren't 4 or 12: ",sequences_not4_not12_pairs_list)
print("there are ",len(unique_load_sequences), " different load sequences in the ", len(train_set_filenames), "sequences in the training set")
#print("unique load seqs", unique_load_sequences)

for key,value in filename_loadseq_dict_seqidkey.iteritems():
    print("key",key,"value",value)
    load_sequence_aslist = str(unique_load_sequences_aslists[value])
    #this arrangement below is necessary because dict has to have a list as its value. 
    #casting the list-cast ndarray as a string means.. parsing it back to numpy requires some creativity. 
    if filename_loadseq_dict_loadlistposkey.get(str(load_sequence_aslist)) != None:
        filename_loadseq_dict_loadlistposkey[str(load_sequence_aslist)].append(key)
    if filename_loadseq_dict_loadlistposkey.get(str(load_sequence_aslist)) == None: 
        content_list = []
        content_list.append(key)
        filename_loadseq_dict_loadlistposkey[str(load_sequence_aslist)] = content_list

print(filename_loadseq_dict_loadlistposkey)
fl_llpk_combined_df = pd.DataFrame.from_dict(filename_loadseq_dict_loadlistposkey,orient='index')
fl_sik_combined_df = pd.DataFrame.from_dict(filename_loadseq_dict_seqidkey,orient='index')
fl_llpk_combined_df.to_csv(analysis_path + "file_loadseq_loadlistkey_COMBINED.csv")
fl_sik_combined_df.to_csv(analysis_path + "file_loadseq_seqid_key_COMBINED.csv")


('train_set size: ', 611)
('sequences with 4-pair repeating units: ', 357)
('sequences WITHOUT 4-pair repeating units: ', ['sequence_1l_413_4_fv1c.npy', 'sequence_1l_411_2_fv1c.npy', 'sequence_3a_561_2_fv1c.npy', 'sequence_3a_690_12_fv1c.npy', 'sequence_3a_715_17_fv1c.npy', 'sequence_3a_600_2_fv1c.npy', 'sequence_3a_664_6_fv1c.npy', 'sequence_3a_646_8_fv1c.npy', 'sequence_1k_337_8_fv1c.npy', 'sequence_3a_618_20_fv1c.npy', 'sequence_3a_570_11_fv1c.npy', 'sequence_1l_394_5_fv1c.npy', 'sequence_1l_404_5_fv1c.npy', 'sequence_3a_703_5_fv1c.npy', 'sequence_1k_321_2_fv1c.npy', 'sequence_3a_584_6_fv1c.npy', 'sequence_1k_292_3_fv1c.npy', 'sequence_1l_371_2_fv1c.npy', 'sequence_1l_402_3_fv1c.npy', 'sequence_3a_650_12_fv1c.npy', 'sequence_1k_344_5_fv1c.npy', 'sequence_1l_401_2_fv1c.npy', 'sequence_3a_718_20_fv1c.npy', 'sequence_3a_661_3_fv1c.npy', 'sequence_1l_364_5_fv1c.npy', 'sequence_1k_357_8_fv1c.npy', 'sequence_1k_341_2_fv1c.npy', 'sequence_1l_397_8_fv1c.npy', 'sequence_3a_582_4_fv1c.npy', '

In [9]:
print(len(unique_load_sequences))
print(unique_load_sequences[0].shape)
for item in unique_load_sequences:
    print(item.shape)

27
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)
(12, 2)


In [10]:
flattened_loadseq_list = []
for item in unique_load_sequences:
    flattened_loadseq_list.append(np.ndarray.flatten(item))
print(flattened_loadseq_list[0].shape,flattened_loadseq_list[-1].shape)

((8,), (24,))


In [11]:
sigma1 = flattened_loadseq_list[0]
sigma2 = flattened_loadseq_list[-1]
print(np.mean(sigma1), np.median(sigma1), np.std(sigma1))
print(flattened_loadseq_list[0])

(7.9344999999999999, 8.0150000000000006, 6.3465211533563801)
[  1.428  14.28   14.28    1.75    1.75   14.28   14.28    1.428]


In [12]:
#flattened_loadseq_list.sort() #doesn't work
#try to pad
flattened_eq_len_loadseq_list = []
long_shape = flattened_loadseq_list[-1].shape
for item in flattened_loadseq_list:
    if item.shape == (8,):
        temp_padded_array = np.zeros(shape=long_shape)
        temp_padded_array[0:8]=item
        temp_padded_array[8:16]=item
        temp_padded_array[16:24]=item
        flattened_eq_len_loadseq_list.append(temp_padded_array)
    if item.shape == (24,):
        flattened_eq_len_loadseq_list.append(item)

# for item in flattened_eq_len_loadseq_list:
#     print(item.shape)
any(item.shape != (24,) for item in flattened_eq_len_loadseq_list)

False

In [13]:
print(str(flattened_eq_len_loadseq_list[0])) # you can cast this as a string. yay. 

[  1.428  14.28   14.28    1.75    1.75   14.28   14.28    1.428   1.428
  14.28   14.28    1.75    1.75   14.28   14.28    1.428   1.428  14.28
  14.28    1.75    1.75   14.28   14.28    1.428]


In [14]:
posn=0
for item in flattened_eq_len_loadseq_list:
    print(posn,np.mean(item), np.median(item), np.std(item))
    posn+=1

(0, 7.9344999999999999, 8.0150000000000006, 6.3465211533563801)
(1, 8.1844999999999999, 7.8540000000000001, 7.1346745370759557)
(2, 8.0845000000000002, 7.6539999999999999, 7.230828496790668)
(3, 7.8539999999999992, 6.8540000000000001, 6.5797778077986795)
(4, 7.8094999999999999, 7.8540000000000001, 6.4708060355723847)
(5, 7.9040000000000008, 6.6539999999999999, 7.0996039326148335)
(6, 7.7469999999999999, 7.8540000000000001, 6.5347522523811099)
(7, 5.5844999999999994, 5.6539999999999999, 4.6965285850295855)
(8, 7.6844999999999999, 7.8540000000000001, 6.5998546006711383)
(9, 5.8969999999999994, 5.6539999999999999, 4.8957927856476937)
(10, 7.0844999999999994, 6.0149999999999997, 5.9689932777646844)
(11, 7.2094999999999994, 7.2649999999999997, 6.0710073916937377)
(12, 7.9076666666666666, 6.8540000000000001, 6.7794900906254663)
(13, 7.8094999999999999, 8.0150000000000006, 6.4740248236059967)
(14, 7.9641666666666664, 6.8540000000000001, 6.722185976715874)
(15, 7.8511666666666668, 8.0150000000

In [15]:
print(flattened_eq_len_loadseq_list[21])

[  1.428  12.28   12.28    1.428   1.428  16.28   16.28    1.428   1.428
  14.28   14.28    1.428   1.428  14.28   14.28    1.428   1.428  12.28
  12.28    1.428   1.428  16.28   16.28    1.428]


In [16]:
#create a dataframe that correlates each sequence (in both sets) to the equal-lengthened array
#fl_llpk_combined_df = pd.DataFrame.from_dict(filename_loadseq_dict_loadlistposkey,orient='index')
#fl_sik_combined_df = pd.DataFrame.from_dict(filename_loadseq_dict_seqidkey,orient='index')
filename_loadseq_dict_seqidkey_equallength = {}
print(filename_loadseq_dict_seqidkey.keys())
print(len(filename_loadseq_dict_seqidkey.keys()))

['sequence_1b_29_10_fv1c', 'sequence_3a_651_13_fv1c.npy', 'sequence_1l_416_7_fv1c.npy', 'sequence_3a_628_10_fv1c', 'sequence_3a_664_6_fv1c.npy', 'sequence_1d_175_16_fv1c.npy', 'sequence_3a_614_16_fv1c.npy', 'sequence_3a_637_19_fv1c.npy', 'sequence_1c_92_3_fv1c.npy', 'sequence_1l_433_4_fv1c.npy', 'sequence_1d_200_11_fv1c.npy', 'sequence_3a_573_14_fv1c.npy', 'sequence_3a_607_9_fv1c', 'sequence_3a_663_5_fv1c.npy', 'sequence_1k_308_9_fv1c', 'sequence_1d_127_8_fv1c.npy', 'sequence_1l_390_1_fv1c.npy', 'sequence_3a_671_13_fv1c.npy', 'sequence_3a_679_1_fv1c.npy', 'sequence_1k_300_1_fv1c.npy', 'sequence_1d_251_2_fv1c.npy', 'sequence_1l_363_4_fv1c.npy', 'sequence_3a_560_1_fv1c.npy', 'sequence_1d_155_26_fv1c.npy', 'sequence_1d_212_23_fv1c.npy', 'sequence_3a_593_15_fv1c.npy', 'sequence_2c_528_9_fv1c', 'sequence_2b_493_4_fv1c.npy', 'sequence_1d_203_14_fv1c.npy', 'sequence_2b_453_4_fv1c.npy', 'sequence_3a_632_14_fv1c.npy', 'sequence_1c_107_8_fv1c.npy', 'sequence_1k_354_5_fv1c.npy', 'sequence_1d_192_

In [17]:
# for key, value in filename_loadseq_dict_seqidkey:
#     filename_loadseq_dict_seqidkey_equallength[key] = 
loadseq_df = pd.DataFrame(flattened_loadseq_list)
loadseq_df.to_csv(analysis_path+"loadseq_flattened.csv")

In [18]:
#fl_sik_test_df file-load sequenceid-key-TEST
#loadseq_loadlist_df
fl_sik_test_df.index.rename(name='seq_name',inplace=True)
fl_sik_test_df.columns = ['loadseq_id']

loadseq_loadlist_df.index.rename(name='loadseq_id',inplace=True)
loadseq_loadlist_df.columns = ['loadseq']


print(fl_sik_test_df.columns,fl_sik_test_df.index)
print(loadseq_loadlist_df.columns,loadseq_loadlist_df.index)
complete_seqname_loadseqlist_df = pd.merge(fl_sik_test_df,loadseq_loadlist_df,how='left',right_index=True,left_on='loadseq_id')
print(complete_seqname_loadseqlist_df.head(2))

(Index([u'loadseq_id'], dtype='object'), Index([u'sequence_1d_168_9_fv1c', u'sequence_1b_29_10_fv1c',
       u'sequence_1c_119_10_fv1c', u'sequence_1b_8_9_fv1c',
       u'sequence_2c_538_9_fv1c', u'sequence_3a_628_10_fv1c',
       u'sequence_2b_469_10_fv1c', u'sequence_1l_398_9_fv1c',
       u'sequence_1b_49_10_fv1c', u'sequence_3a_687_9_fv1c',
       ...
       u'sequence_1b_38_9_fv1c', u'sequence_1c_68_9_fv1c',
       u'sequence_1d_198_9_fv1c', u'sequence_1k_359_10_fv1c',
       u'sequence_1k_319_10_fv1c', u'sequence_2b_448_9_fv1c',
       u'sequence_1d_229_10_fv1c', u'sequence_1l_428_9_fv1c',
       u'sequence_1d_259_10_fv1c', u'sequence_1l_388_9_fv1c'],
      dtype='object', name=u'seq_name', length=108))
(Index([u'loadseq'], dtype='object'), RangeIndex(start=0, stop=27, step=1, name=u'loadseq_id'))
                        loadseq_id  \
seq_name                             
sequence_1d_168_9_fv1c           0   
sequence_1b_29_10_fv1c           0   

                                

In [19]:
complete_seqname_loadseqlist_df.to_csv(analysis_path+"complete_seqname_loadseqlist.csv")